## Importing Libraries

In [ ]:
import requests
import io
import zipfile
import re
import pandas as pd
import numpy as np
import json
import modules.psql as psql

In [ ]:
%run config_psql.ipynb

In [ ]:
# Settings configurations

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
url = "https://cricsheet.org/downloads/recently_played_30_json.zip"
filetype = ".json"

In [ ]:
response = requests.get(url)

if response.status_code == 200:
    content = response.content
    
    zip_file = zipfile.ZipFile(io.BytesIO(content))
    
    with zip_file.open('README.txt') as f:
        lines = [line.decode('utf-8') for line in f.readlines()]
        pattern = re.compile(r'(\d{4}-\d{2}-\d{2}) - ([^-]+) - ([^-]+) - (\w+) - (\d+) - (.+)')
        ids = [match.group(5) for line in lines if (match := pattern.match(line))]
    f.close()

In [ ]:
df_meta = pd.DataFrame()
df_match = pd.DataFrame()
df_official = pd.DataFrame()
df_registry = pd.DataFrame()
df_player_def = pd.DataFrame()
df_innings = pd.DataFrame()
df_deliveries = pd.DataFrame()
df_powerplay = pd.DataFrame()
df_absent_hurt_def = pd.DataFrame()
df_miscounted_overs_def = pd.DataFrame()

In [ ]:
print(len(ids), " files present")
for file in ids:
    with zip_file.open(file+filetype) as jsonfile:
        data = json.load(jsonfile)
      
        # DataFrame to store - Metadata
        df_meta = pd.concat([df_meta, pd.DataFrame([data["meta"]]).assign(filename=file, filetype=filetype)])
        df_meta['created'] = pd.to_datetime(df_meta['created'])
        
        # DataFrame to store - match details
        df_info = pd.DataFrame([data["info"]])
        df_match_temp = pd.concat([
            pd.json_normalize(df_info['event'], sep='_').add_prefix('event_'),
            pd.DataFrame(df_info[list(set(['balls_per_over','season', 'gender', 'city', 'venue', 'match_type', 'match_type_number', 'overs', 'team_type']) & set(df_info.columns))]),
            df_info['dates'].apply(lambda x: [x[0], x[-1]]).apply(pd.Series).rename(columns={0: 'start_date', 1: 'end_date'}),
            df_info['teams'].apply(lambda x: [x[0], x[1]]).apply(pd.Series).rename(columns={0: 'team_host', 1: 'team_visitor'}),
            pd.json_normalize(df_info['toss'], sep='_').add_prefix('toss_'),
            pd.json_normalize(df_info['outcome'], sep='_').add_prefix('outcome_')
        ], axis=1).assign(match_id = file)
        if 'player_of_match' in df_info.columns:
            df_match_temp['player_of_match'] = df_info['player_of_match'].apply(lambda x: ','.join(x))
            
        df_match = pd.concat([df_match, df_match_temp])
            
        # DataFrame to store official details
        df_umpire = pd.json_normalize(df_info['officials'], sep = '_')

        umpire_set = set()
        for column in df_umpire.columns:
            umpire_set.update(df_umpire[column].explode().dropna())

        df_umpire2 = pd.DataFrame(index=list(umpire_set), columns=df_umpire.columns).fillna(False)
        for column in df_umpire.columns:
            df_umpire2[column] = df_umpire2.index.isin(df_umpire[column].explode().dropna())
        df_umpire2 = df_umpire2.reset_index().rename(columns={'index': 'name'}).assign(match_id = file)     
        
        df_official = pd.concat([df_official, df_umpire2])

        # DataFrame to store - registry details
        df_registry = pd.DataFrame(list(data["info"]["registry"]["people"].items()), columns=['people', 'identifier']).assign(match_id = file)

        # DataFrame to store - match player details
        df_player = pd.json_normalize(df_info['players']).melt(var_name='team', value_name='player').explode('player').assign(match_id = file)
        
        # DataFrame to store - innings ball-by-ball details
        df_innings = pd.concat([df_innings, pd.json_normalize(data['innings'], sep = '_').drop('overs', axis = 1).assign(match_id = file)])
        df_deliveries = pd.concat([df_deliveries, pd.json_normalize(data['innings'], record_path=['overs', 'deliveries'], meta=['team',['overs', 'over']], sep='_').assign(match_id = file)])
        
        # DataFrame to store - powerplay details
        for i in data['innings']:
            if 'powerplays' in pd.json_normalize(i).columns:
                index = data['innings'].index(i)
                df_powerplay = pd.concat([df_powerplay, pd.json_normalize(data['innings'][index], record_path = ['powerplays'], meta = ['team'], sep = '_').assign(match_id = file)])
           
        if 'absent_hurt' in df_innings.columns:
            df_absent_hurt = df_innings[['team','absent_hurt']].explode('absent_hurt').assign(match_id = file)
        df_miscounted_overs = pd.DataFrame([
            {
                "team": inning.get("team", ""),
                "miscounted_over": over_number,
                "balls": over_data.get("balls", ""),
                "umpire": over_data.get("umpire", "")
            }
            for inning in data.get("innings", [])
            for over_number, over_data in inning.get("miscounted_overs", {}).items()
        ])
    print(file + " executed!")

In [ ]:
# Merging registry details into match-player details
df_player.reset_index(inplace = True, drop = True)
df_player.rename(columns = {'player':'name'}, inplace = True)
df_player['player_id'] = df_player.merge(df_registry, how='left', left_on=['match_id', 'name'], right_on=['match_id', 'people'])['identifier']

In [ ]:
# Upsert MetaData information
query = psql.upsert(
    engine,
    dataFrame = df_meta,
    table = "meta",
    schema = "dwh",
    pk_col = list(df_meta.columns),
    update_col = list(df_meta.columns))

In [ ]:
# Load player-match information
match_id_list = ", ".join([f"'{match_id}'" for match_id in ids])

with engine.connect() as conn:
    conn.execute(f"DELETE FROM dwh.match_player WHERE match_id IN ({match_id_list})")
    
count_rows = df_player.to_sql('match_player', schema='dwh', con=engine, if_exists='append', method='multi', index=False)

with engine.connect() as conn:
    conn.execute("""
        UPDATE dwh.match_player MP
        SET is_registered = FALSE
        FROM dwh.people P
        WHERE MP.player_id = P.identifier AND P.identifier IS NULL;
    """)

In [ ]:
# Load match information into Stage table
with engine.connect() as conn:
    conn.execute("TRUNCATE TABLE stg.match")

count_rows = df_match.to_sql('match', schema = 'stg', con = engine, if_exists='append', method = 'multi', index = False)

In [ ]:
with engine.connect() as conn:
    conn.execution_options(isolation_level = "AUTOCOMMIT")
    with conn.begin():
        conn.execute("CALL dwh.LoadMatch()")